### Implementing Regression problem for the same dataset, response variable is Estimatedsalary

In [1]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

### Data preprocessing

In [2]:
data = pd.read_csv('Churn_Modelling.csv')

data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data.drop(columns=['RowNumber', "CustomerId", 'Surname'], inplace=True)

In [4]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
label_encoder_gender = LabelEncoder()

In [6]:
data['Gender'] = label_encoder_gender.fit_transform(data["Gender"])

In [7]:
one_hot_encoder_geography = OneHotEncoder(sparse_output=False)

geography_reshape = data['Geography'].values.reshape(-1, 1)

geography_encoder = one_hot_encoder_geography.fit_transform(geography_reshape)

geography_df = pd.DataFrame(geography_encoder, columns=one_hot_encoder_geography.get_feature_names_out(['Geography']), index = data.index)

geography_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
data.drop(columns=["Geography"], inplace=True)

In [9]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,42,2,0.00,1,1,1,101348.88,1
1,608,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,39,1,0.00,2,0,0,93826.63,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0


In [10]:
data = pd.concat([data, geography_df], axis= 1)

data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
data['EstimatedSalary']

0       101348.88
1       112542.58
2       113931.57
3        93826.63
4        79084.10
          ...    
9995     96270.64
9996    101699.77
9997     42085.58
9998     92888.52
9999     38190.78
Name: EstimatedSalary, Length: 10000, dtype: float64

In [12]:
X = data.drop(columns=['EstimatedSalary'])

y = data['EstimatedSalary']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

### Save your encoder and scaler files as pickle file

In [14]:
with open('label_encoder_regression_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('one_hot_encoder_regression_geography.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_geography, file)

with open('scaler_regression.pkl', 'wb') as file:
    pickle.dump(scaler, file)

### Model Training

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

2025-01-12 12:56:54.676266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-12 12:56:54.693570: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-12 12:56:54.698474: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-12 12:56:54.710411: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
model = Sequential(
    [
        Input(shape=(X_train.shape[1],)), 
        Dense(64, activation='relu'),
        Dense(32, activation = 'relu'),
        Dense(1)
    ]
)

2025-01-12 12:56:57.045304: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-01-12 12:56:57.045328: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:135] retrieving CUDA diagnostic information for host: aljebra
2025-01-12 12:56:57.045333: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:142] hostname: aljebra
2025-01-12 12:56:57.045441: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:166] libcuda reported version is: 535.183.1
2025-01-12 12:56:57.045457: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:170] kernel reported version is: 535.183.1
2025-01-12 12:56:57.045462: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:249] kernel version seems to match DSO: 535.183.1


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

### forward and backward pass

In [18]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.MeanSquaredError()

In [19]:
model.compile(optimizer = optimizer, loss = loss , metrics=['mse'])

### Set the Tensorboard for callbacks and EarlyStopping

In [20]:
log_dir = "Regression_logs/fit/" + datetime.datetime.now().strftime("%Y/%m/%d, %H:%M:%S")

tensorflow_callbacks = TensorBoard(log_dir=log_dir, histogram_freq= 1)

In [21]:
early_stopping_callback = EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=10)

In [22]:
history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test), epochs = 50,
    callbacks = [tensorflow_callbacks, early_stopping_callback]
)

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 12144325632.0000 - mse: 12144325632.0000 - val_loss: 3498766848.0000 - val_mse: 3498766848.0000
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3404802048.0000 - mse: 3404802048.0000 - val_loss: 3396491776.0000 - val_mse: 3396491776.0000
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3349297920.0000 - mse: 3349297920.0000 - val_loss: 3371219712.0000 - val_mse: 3371219712.0000
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3372855296.0000 - mse: 3372855296.0000 - val_loss: 3363414272.0000 - val_mse: 3363414272.0000
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3344268288.0000 - mse: 3344268288.0000 - val_loss: 3392342528.0000 - val_mse: 3392342528.0000
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3298042368.0000 - mse: 3298042368.0000 - val_loss: 3356802560.0000 - val_mse: 3356802560.0000
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3365984256.000

### Visualize your training output

In [23]:
%load_ext tensorboard

In [24]:
%tensorboard --logdir Regression_logs/fit/

### Evaluate loss on test data

In [25]:
test_loss, test_mse = model.evaluate(X_test, y_test)

print("Model error : ", test_mse)
print("model loss : ", test_loss)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3437301248.0000 - mse: 3437301248.0000  
Model error :  3355535616.0
model loss :  3355535616.0


In [26]:
model.save('regression_model.h5')